In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Ignore python warnings

### Import the dependencies

In [ ]:
import numpy as np
from sklearn.utils import class_weight

### Get the Data

In [ ]:
## Create directories for Keras ImageDataGenerator.flow_from_directory
train_dir_abnormal = "./train_data/abnormal"
train_dir_normal = "./train_data/normal"

valid_dir_abnormal = "./valid_data/abnormal"
valid_dir_normal = "./valid_data/normal"

In [ ]:
from src.data.dataloader import create_dir, load_images

In [ ]:
create_dir(train_dir_abnormal)
create_dir(train_dir_normal)

In [ ]:
create_dir(valid_dir_abnormal)
create_dir(valid_dir_normal)

In [ ]:
# Load Train Images
images_path = 'MURA-v1.1/train_image_paths.csv'
train_image_paths = load_images(images_path)
print("Training Set Images loaded!")

In [ ]:
# Load Validation set images
images_path = 'MURA-v1.1/valid_image_paths.csv'
train_image_paths = load_images(images_path)
print("Validation Set Images loaded!")

### Create the Architectures

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import (ModelCheckpoint, TensorBoard)

from src.model.xception import get_xception
from src.model.dense169 import get_dense169

#### Hyperparameters

In [ ]:
input_shape = 320
batch_size = 8
epochs = 10
learning_rate = 0.0001

#### Dense169 Model

In [ ]:
from keras.applications.densenet import preprocess_input
dense169_mura_single = get_dense169(input_shape, learning_rate)

#### Get Xception V2

In [ ]:
from keras.applications.xception import preprocess_input
xception_mura_single = get_xception(input_shape, learning_rate)

### Start Training

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=30,
        horizontal_flip=True,
        preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'train_data/',
        target_size=(input_shape, input_shape),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input )

valid_generator = valid_datagen.flow_from_directory(
        'validation_data',
        target_size=(input_shape, input_shape),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False)

In [ ]:
## Compute Class Weights
weights = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)

In [ ]:
filepath="xception_v2-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"  # Change the name based on which model is being trained

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=False)

callbacks_list = [checkpoint, tensorboard]

In [ ]:
training_data_size = len(train_generator.filenames)
validation_data_size = len(valid_generator.filenames)

print("Number of Training examples: ", training_data_size)
print("Number of Validation examples: ", validation_data_size)

#### Train Dense169

In [ ]:
dense169_mura_single.fit_generator(train_generator,
                        validation_data=valid_generator,
                        steps_per_epoch=training_data_size // batch_size,
                        class_weight=weights,
                        callbacks=callbacks_list, 
                        validation_steps=validation_data_size // batch_size,
                        epochs=epochs)

#### Train Xception

In [ ]:
xception_mura_single.fit_generator(train_generator,
                        validation_data=valid_generator,
                        steps_per_epoch=training_data_size // batch_size,
                        class_weight=weights,
                        callbacks=callbacks_list, 
                        validation_steps=validation_data_size // batch_size,
                        epochs=epochs)